In [1]:
import pandas as pd
import numpy as np
import os
import sys

In [2]:
# helper function

def find_n_nearest_euclid(n, data, index, index_fips_dict):
    index_row = data[index]
    n_nearest = [float(sys.maxsize)] * n
    n_nearest_counties = []
    for i in range(0, len(data)):
        if i != index:
            curr = data[i]
            county = index_fips_dict[i]
            dist = 0.0
            for j in range(0, len(index_row)):
                dist += (index_row[j] - curr[j]) ** 2
            
            if len(n_nearest) == 0:
                n_nearest.append(dist)
                n_nearest_counties.append(county)
            else:
                inserted = False
                j = 0
                while inserted == False and j < n:
                    if dist < n_nearest[j]:
                        first = n_nearest[:j]
                        middle = [dist]
                        last = n_nearest[j:n-1]
                        n_nearest = first + middle + last
                        
                        first_counties = n_nearest_counties[:j]
                        middle_counties = [county]
                        last_counties = n_nearest_counties[j:n-1]
                        n_nearest_counties = first_counties + middle_counties + last_counties
                        
                        inserted = True
                    j += 1
    
    return n_nearest_counties

In [5]:
# function finding the n nearest counties in order
# parameters: 
# 1. n an integer
# 2. pca 0 or 1 indicating if you don't or do use the pca data
# 3. data_type a string the subset of the data 

def n_nearest_euclid(n, fips, pca, data_type):
    # get the file name and load the file
    data_file = ''
    if pca == 1:
        data_file += 'pca_'
    else:
        data_file += 'data_'
    data_file += str.lower(data_type)
    data_file += '.csv'
    df = pd.read_csv(data_file)
    
    # get the dict of indices vs fips
    index_fips = []
    index_fips.append(list(df['Unnamed: 0']))
    index_fips.append(list(df['FIPS']))
    index_fips_dict = {}
    index_dict = {}
    for i in range(0, len(index_fips[0])):
        index_fips_dict[index_fips[1][i]] = index_fips[0][i]
        index_dict[index_fips[0][i]] = index_fips[1][i]
    
    # get the n nearest
    df.drop(columns = ['Unnamed: 0', 'FIPS'])
    data = df.to_numpy()
    index = index_fips_dict[fips]
    n_nearest = find_n_nearest_euclid(n, data, index, index_dict)
    return n_nearest

In [9]:
n_nearest_euclid(6, 1007, 1, 'geography')

[1009, 1001, 1021, 1015, 1003, 1013]